In [6]:
import os
import sys
import json
from rv_zep import parse_file


In [13]:
config_file = open("./unit_config.json")
units = json.load(config_file)

In [15]:
units['fint']['number']

1

In [14]:
for unit in units:
    print(unit)

fint
fmult
fadd
fdiv


In [ ]:
class ScoreBoard:
    def __init__(self, available_ops, units_configuration):

        self.units_configuration = units_configuration
        self.instruction_queue = []
        self.board = []
        self.ops = {}
        self.clock_counter = 0
        self.solution = self.build_solution(self.instruction_queue)
    
    def tick(self):

        issue_instruction()
        update_board()
        self.clock_counter += 1

        if self.instruction_queue == [] and not board_busy():
            return True
        else:
            return False

    def build_board(self):

        for unit in self.units_configuration:
            unit_number = self.units_configuration[unit]['number']
        ...

    def build_solution(self):
        instruction = 0
        issue = 1
        read = 2
        ex = 3
        write = 4

        return solution

    
    def update_board(self):
        [name][busy][op][fi][fj][fk][qj][qk][rj][rk][instruction_line][state][state_cycles]
        ...
    
    def issue_instruction(self):

        instruction = self.instruction_queue.peek
        
        #check if there are free FUs available
        #check if no other issues write to the same destination

        ...

    def board_busy(self):
        busy = False
        for unit in self.board:
            busy = unit['busy'] or busy
        return busy


    def print_board(self):
        ...

    def print_solution(self):
        ...

    def run(self):
        
        self.done = False
        while not self.done:
            self.tick()
        self.print_solution()
        


class FunctionalUnit:
    def

In [18]:
queue[0]['opcode']

0

In [16]:
queue =parse_file('example.s')